<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# GitHub - Send templates maintainer monthly report
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/GitHub/GitHub_Send_templates_created_on_a_notebooks_to_Slack_channel.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://bit.ly/3JyWIk6">Give Feedbacks</a> | <a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas/Naas_Start_data_product.ipynb" target="_parent">Generate Data Product</a>

**Tags:** #github #issues #merged #rest #api #snippet #operations #email

**Author:** [Benjamin Filly](https://www.linkedin.com/in/benjamin-filly-05427727a/)

**Last update:** 2023-07-24 (Created: 2023-07-24)

**Description:** This notebook demonstrates how to send the issues merged on GitHub this month  to an e-mail. It includes the sections below:

- 🔃 **Templates merged this month:** a table and the issues merged this month.
- 📈 **The estimate of this month:** the number of estimate you got this month 

*NB: Execution time may takes between 1 to 3 min.*

**References:**
- [GitHub REST API Documentation](https://docs.github.com/en/rest/issues/issues?apiVersion=2022-11-28#update-an-issue)
- [os](https://docs.python.org/3/library/os.html)
- [json](https://docs.python.org/3/library/json.html)
- [datetime](https://docs.python.org/3/library/datetime.html)
- [pandas](https://pandas.pydata.org/docs/)
- [requests](https://docs.python-requests.org/en/latest/)
- [slack (naas_drivers)](https://pypi.org/project/naas-drivers/)

## Input

### Import libraries

In [ ]:
import pandas as pd
from github import Github
from datetime import datetime, timezone
import requests
from bs4 import BeautifulSoup
from IPython.display import display, HTML
import numpy as np
from datetime import datetime
import naas
import naas_drivers
import warnings
warnings.filterwarnings("ignore")

### Setup Variables
- `github_token`: [GitHub token](https://help.github.com/en/github/authenticating-to-github/creating-a-personal-access-token-for-the-command-line)
- `repo_name`: name of the repository in two part: owner_name/repository_name
- `contributor_profile`: GitHub username of the contributor
- `url`:Url of the project where the estimated are located
- `cron`: cron params for naas scheduler change it using [Crontab](https://crontab.guru/)
- `email_to`: List to email addresses of the receiver(s)
- `email_from`:  Email sender: Replace with your email account or notification@naas.ai
- `subject`: Email subject


In [ ]:
#Input
github_token = naas.secret.get("GITHUB_TOKEN") or "YOUR_GITHUB_TOKEN"
repo_name = "jupyter-naas/awesome-notebooks" #Example: jupyter-naas/awesome-notebooks
contributor_profile = "USERNAME" 
url = "https://github.com/orgs/jupyter-naas/projects/10/views/20" #example: https://github.com/orgs/jupyter-naas/projects/10/views/20
cron = "0 0 28-31 * *" # At 00:00 on every day-of-month from 28 through 31

#Output
email_to = ["example@example.com"]
email_from = "notification@naas.ai"
subject = "Templates Maintainer Monthly Report"

## Model

### Connect to GitHub repo

In [ ]:
# Connect to the GitHub API using pygithub library
g = Github(github_token)

# Get the repository
repo = g.get_repo(repo_name)

### Get PRs merged
So here we're filtering out all merged PRs on the `contributor_profile`

In [ ]:
# Function to check if a date is within the current month
def is_date_in_current_month(date_obj):
    current_date = datetime.now(timezone.utc)
    return date_obj.year == current_date.year and date_obj.month == current_date.month

# Get the merged PRs
pull_requests = repo.get_pulls(state='closed', sort='updated', direction='desc')
print("✅ Pull Requests fetched:", pull_requests.totalCount)

# Filter merged pull requests during this month and assigned to the specific contributor
merged_assigned_pull_requests = [pr for pr in pull_requests if pr.assignee and pr.assignee.login == contributor_profile and pr.merged_at and is_date_in_current_month(pr.merged_at)]

# Print the merged PR count for the specific contributor during this month
print(f"📌Number of assigned merged PRs in current month: {len(merged_assigned_pull_requests)}")

### Get templates added by PRs closed

In [ ]:
# Store the relevant information in a DataFrame
data = []
for pr in merged_assigned_pull_requests:
    tmp = {
        "title": pr.title,
        "number": pr.number,
        "url": pr.html_url,
        "assignee": pr.assignee.login,
        "created_at": pr.created_at.isoformat(),
        "updated_at": pr.updated_at.isoformat(),
        "merged_at": pr.merged_at.isoformat(),
    }
    data.append(tmp)
print("Number of merged PR:", len(merged_assigned_pull_requests))
df_pr = pd.DataFrame(data)
df_pr.head(3)

### Get Data from project view
This function returns organised data from the project view soup using BeautifulSoup.

In [ ]:
# Init
data_bs4 = []

# Get HTML from URL
response = requests.get(url)
html = response.text

# Parse HTML
soup = BeautifulSoup(html, "html.parser")

# Get cards
elements = soup.find_all("script", {"id": "memex-items-data"})

# Iterate over the elements and split their text
for element in elements:
    text = element.text
    split_text = text.split('{"contentId":')[1:]  # Split the text as needed
    
    # Split the soup for each element
    for s in split_text:
        s = s.split('"memexProjectColumnId":')[1:]
        # Get the values using splits
        title = s[0].split('"raw":"')[-1].split('"')[0]
        issue_number = s[0].split('"number":')[-1].split(',')[0]
        assignees = s[1].split('"login":"')[-1].split('"')[0]
        PR_url = s[2].split('"url":"')[-1].split('"')[0]
        estimate = s[3].split('"value":')[-1].split('}')[0]
        
        # Handle possible error
        if not str(issue_number).isdigit():
            issue_number = "❌ Error"
        
        # Create a dictionary with the values
        tmp_bs4 = {
            "Title": title,
            "Issue Number": issue_number,
            "Assignees": assignees,
            "PR URL": PR_url,
            "Estimate": estimate,
        }
        # Append the dictionary to the data list
        data_bs4.append(tmp_bs4)

        
print("Number of issues:", len(split_text))
# Create a DataFrame from the data list
df_init = pd.DataFrame(data_bs4)
df_init.head(3)

### Calculate estimates
I've filtered the issues using the PR URLs

In [ ]:
# Filter df_init with the URLs from df_pr
filtered_df_init = df_init[df_init["PR URL"].isin(df_pr["url"])]
filtered_df_init['Estimate'] = filtered_df_init['Estimate'].str.replace("null", "0")


# Set the display option for max column width to ensure the link is fully displayed
pd.set_option('display.max_colwidth', None)

# Convert the "PR URL" column values into clickable links using HTML formatting
filtered_df_init["PR URL"] = filtered_df_init["PR URL"].apply(lambda x: f'<a href="{x}">{x}</a>')

# Calculate the total of the "Estimate" column and convert it to an integer
total_estimate = int(filtered_df_init["Estimate"].astype(float).sum())

# Display the DataFrame with clickable links and format the "Estimate" column as integers
filtered_df_init["Estimate"] = filtered_df_init["Estimate"].astype(float).astype(int)
display(HTML(filtered_df_init.to_html(escape=False, index=False)))

# Separate sentence with emoji for the total estimate
print(f"\n🚀 The total estimate is: {total_estimate}")

### Creating the E-mail
We used `Naas_emailbuilder_demo.ipynb` to create our email.

In [ ]:
# Sample logo URL
logo_img = "https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=186"

# Get the current date
current_date = datetime.now().strftime("%B %Y")

# Center-align the content of the email
center_style = 'style="text-align: center;"'

# Convert the "Estimate" column values to string
filtered_df_init["Estimate"] = filtered_df_init["Estimate"].astype(str)

# Define the content for the email using the email builder
email_content = {
    "element": naas_drivers.emailbuilder.title(f'<div {center_style}>📝 Report on templates created for {contributor_profile}</div>'),
    "heading": naas_drivers.emailbuilder.heading(f'<div {center_style}>Monthly Report - {current_date}</div>'),
    "text": naas_drivers.emailbuilder.text(f'<div {center_style}>Here is the summary of your contributions for this month</div>'),
    "table": naas_drivers.emailbuilder.table(filtered_df_init, header=True, border=True),
    "total_estimate": naas_drivers.emailbuilder.text(f'<div {center_style}><b>Total Estimate: {total_estimate}</b></div>'),
    "footer": naas_drivers.emailbuilder.footer_company(
        networks=[{"img_src": logo_img, "href": ""}],
        company=["Company Informations"],
        legal=["Legal Informations"],
    ),
}

# Generate the email content as HTML
content = naas_drivers.emailbuilder.generate(display="iframe", **email_content)

## Output

### Send email

In [ ]:
# Send the email using naas.notification
naas.notification.send(email_to=email_to, subject=subject, html=content, email_from=email_from)

### Add scheduler

In [ ]:
naas.scheduler.add(cron=cron)